<a href="https://colab.research.google.com/github/aboagyeso/v2_code/blob/master/Copy_of_P42345_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Embedding, Conv1D, MaxPooling1D, GRU
from keras import regularizers
from keras.optimizers import SGD
from keras.optimizers import Adam, RMSprop
from keras.layers import LSTM
import numpy
import csv
import pandas as pd
import hashlib
import random 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

seed = 7
numpy.random.seed(seed)

maxlen = 100
batch_size = 128

dataset = pd.read_csv("https://raw.githubusercontent.com/aboagyeso/LSTM_SMILES/master/Data/smiles_training/v2/P42345.csv", delimiter=",")
#X_train = dataset.iloc[:,0:1].values
#y_train = dataset.iloc[:,1:2].values
X_train = dataset[['smiles']].values
y_train = dataset[['pAc']].values

for p in range (X_train.shape[0]):
  s = X_train[p,0]
  s = s.replace("[nH]","A")
  s = s.replace("Cl","L")
  s = s.replace("Br","R")
  s = s.replace("[C@]","C")
  s = s.replace("[C@@]","C")
  s = s.replace("[C@@H]","C")
  s =[s[i:i+1] for i in range(0,len(s),1)]
  s = " ".join(s)
  X_train[p,0] = s
X_train = X_train[:,0]  
#y_train = y_train[:,0]
X_train = X_train.tolist()


tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=100)
print(X_train)


Using TensorFlow backend.


[[0 0 0 ... 8 8 2]
 [0 0 0 ... 3 1 4]
 [0 0 0 ... 4 1 3]
 ...
 [0 0 0 ... 4 1 1]
 [0 0 0 ... 1 1 3]
 [0 0 0 ... 1 1 3]]


In [2]:
dataset = pd.read_csv("https://raw.githubusercontent.com/aboagyeso/LSTM_SMILES/master/Data/smiles_validation/smilesP42345.csv", delimiter=",")
#X_test = dataset.iloc[:,0:1].values
#y_test = dataset.iloc[:,1:2].values

X_test = dataset[['smiles']].values
y_test = dataset[['pAc']].values




#X_test = dataset.iloc[:,0:1].values
#y_test = dataset.iloc[:,1:2].values

for p in range (X_test.shape[0]):
  s = X_test[p,0]
  s = s.replace("[nH]","A")
  s = s.replace("Cl","L")
  s = s.replace("Br","R")
  s = s.replace("[C@]","C")
  s = s.replace("[C@@]","C")
  s = s.replace("[C@@H]","C")
  s =[s[i:i+1] for i in range(0,len(s),1)]
  s = " ".join(s)
  X_test[p,0] = s
X_test = X_test[:,0]  
#y_test = y_test[:,0]
X_test = X_test.tolist()
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=100)
#print(X_test)
#print(X_test.shape)



model = Sequential()
model.add(Embedding(100, 128, input_length=100))
model.add(LSTM(128))
model.add(Dense(100, activation='tanh'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer=Adam(0.001))


model.fit(X_train, y_train,
          batch_size=128,
          epochs=20)



W0728 01:07:41.297446 140533595686784 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0728 01:07:41.341341 140533595686784 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0728 01:07:41.353848 140533595686784 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0728 01:07:41.699548 140533595686784 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0728 01:07:41.967604 140533595686784 deprecation.py:323

Epoch 1/20
27670/27670 [==============================] - 139s 5ms/step - loss: 3.6311
Epoch 2/20
27670/27670 [==============================] - 219s 8ms/step - loss: 1.3983
Epoch 3/20
27670/27670 [==============================] - 218s 8ms/step - loss: 1.3828
Epoch 4/20
27670/27670 [==============================] - 218s 8ms/step - loss: 1.3718
Epoch 5/20
27670/27670 [==============================] - 217s 8ms/step - loss: 1.2304
Epoch 6/20
27670/27670 [==============================] - 217s 8ms/step - loss: 1.0572
Epoch 7/20
27670/27670 [==============================] - 217s 8ms/step - loss: 1.0223
Epoch 8/20
27670/27670 [==============================] - 218s 8ms/step - loss: 1.0080
Epoch 9/20
27670/27670 [==============================] - 217s 8ms/step - loss: 0.9948
Epoch 10/20
27670/27670 [==============================] - 167s 6ms/step - loss: 0.9808
Epoch 11/20
27670/27670 [==============================] - 136s 5ms/step - loss: 0.9661
Epoch 12/20
27670/27670 [================

In [3]:
score = model.evaluate(X_test, y_test,
                            batch_size=128)
print('Test score:', score)

predictions = model.predict(X_test)


mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2_score = r2_score(y_test, predictions)

print(str(mae)+"\t"+str(mse)+"\t"+str(r2_score))

258/258 [==============================] - 0s 1ms/step
Test score: 0.4842074869215026
0.5497087207649618	0.4842074877670984	0.6469425493397122
